In [ ]:
import sys
FOLDER_PATH = '/content/drive/MyDrive/Colab Notebooks/porject/'
sys.path.append(FOLDER_PATH)

In [ ]:
!pip install -r drive/MyDrive/Colab\ Notebooks/porject/requirements

In [ ]:
import re
import imp
import time

import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

from pandarallel import pandarallel
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer		
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
pandarallel.initialize(progress_bar=False)

import pymorphy2
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
from string import punctuation

# pd.set_option('display.max_colwidth', None)
import help_functions
imp.reload(help_functions)
from help_functions import is_full_phone, full_phone, is_short_phone, short_phone
from help_functions import num_words_count, digits_count, contact_words_count, count_emoji
from help_functions import  tel_count, unique_words, preprocess_text, preprocess_text_light

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

In [41]:
df = pd.read_csv('./drive/MyDrive/Colab Notebooks/porject/_data/train.csv')
df_test = pd.read_csv('./drive/MyDrive/Colab Notebooks/porject/_data/val.csv')

In [42]:
df.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - еврокнижка. Размер спального места 640-200. Все подушки и подлокотники съёмные и мобильные. Ящик для белья. Продаем в связи с переездом. Муж может помочь с погрузкой. Небольшой торг возможен. Звоните в любое время.,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0
1,Кожух рулевой колонки Даф хф 91 4509834,"Кожух рулевой колонки DAF XF 94 (60066004)/\n /\nартикул 2309862/\n /\nВ наличии на складе./\n /\nНомер детали 262620 сообщайте при бронировании./\n /\nУважаемые пользователи, в данный момент у нашей компании нет возможности оперативно отвечать на Ваши сообщения внутри Авито, в связи с чем данная функция отключена. По всем вопросам обращайтесь по телефону указанному в объявлении./\n/\nГрузовая разборка запчасти б/у./\n /\nВ наличии любые другие запчасти для грузовых Даф/\n /\nЕсть запчасти на все модели./\n/\nУ нас большой склад в России и Европе, поэтому звоните уточняйте по наличию./\n /\nПри необходимости отправляем запчасти в любые регионы, гарантия на запчасти 2 месяц!/\n /\nЗвоните, скажите что Вы с Авито и мы дадим скидку 4%, точно договоримся!",Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! /\nОтвечу сразу на вопрос: почему здесь?/\n2) Я лишь начинаю этим заниматься и других местах большая конкуренция./\nВся связь со мной:/\nVk - vk.com/id153740153/\nDiscord - Goku#4407,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1
3,Телевизор sharp.Смарт тв.Интернет,"Продам телевизор . Диагональ 450.наличие входа USB для подключения накопителей и специализированных многофункциональных устройств;/\n/\nфункция приема сигнала сети Интернет;/\n/\nвозможность останавливать трансляцию ТВ-программ при помощи Time Shift;/\n/\nтехнология Quattron, улучшающая цветопередачу экрана;/\n/\nсканирующая подсветка, не допускающая смазывания кадров;/\n/\nимеющаяся функция Skype Ready;/\n/\nкомплектование жидкокристаллическими панелями последнего поколения X-gen;/\n/\nсистема Aquos.",Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в технике скрапбукинг,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0


In [43]:
df['id'] = ['id_{}'.format(str(el).zfill(7)) 
            for el in np.arange(len(df))]
df_test['id'] = ['id_{}'.format(str(el).zfill(7)) 
                 for el in np.arange(len(df), len(df) + len(df_test))]

In [44]:
%%time 
df['is_empty_price'] = df['price'].isna().astype('int64')
df['month'] = pd.to_datetime(df['datetime_submitted']).dt.month
df['hour'] = pd.to_datetime(df['datetime_submitted']).dt.hour
df['is_full_phone'] = df['description'].parallel_apply(lambda x: is_full_phone(x))
df['full_phone'] = df['description'].parallel_apply(lambda x: full_phone(x))
df['is_short_phone'] = df['description'].parallel_apply(lambda x: is_short_phone(x))
df['short_phone'] = df['description'].parallel_apply(lambda x: short_phone(x))
df['num_words_count'] = df['description'].parallel_apply(lambda x: num_words_count(x))
df['digits_count'] = df['description'].parallel_apply(lambda x: digits_count(x))
df['contact_words_count'] = df['description'].parallel_apply(lambda x: contact_words_count(x))
df['tel_count'] = df['description'].parallel_apply(lambda x: tel_count(x))
df['count_emoji'] = df['description'].parallel_apply(lambda x: count_emoji(x))

df['title_prep_l'] = df['title'].parallel_apply(lambda x: preprocess_text_light(x))
df['desc_prep_l'] = df['description'].parallel_apply(lambda x: preprocess_text_light(x))

CPU times: user 26.3 s, sys: 27 s, total: 53.2 s
Wall time: 9min 30s


In [45]:
%%time 
df_test['is_empty_price'] = df_test['price'].isna().astype('int64')
df_test['month'] = pd.to_datetime(df_test['datetime_submitted']).dt.month
df_test['hour'] = pd.to_datetime(df_test['datetime_submitted']).dt.hour
df_test['is_full_phone'] = df_test['description'].parallel_apply(lambda x: is_full_phone(x))
df_test['full_phone'] = df_test['description'].parallel_apply(lambda x: full_phone(x))
df_test['is_short_phone'] = df_test['description'].parallel_apply(lambda x: is_short_phone(x))
df_test['short_phone'] = df_test['description'].parallel_apply(lambda x: short_phone(x))
df_test['num_words_count'] = df_test['description'].parallel_apply(lambda x: num_words_count(x))
df_test['digits_count'] = df_test['description'].parallel_apply(lambda x: digits_count(x))
df_test['contact_words_count'] = df_test['description'].parallel_apply(lambda x: contact_words_count(x))
df_test['tel_count'] = df_test['description'].parallel_apply(lambda x: tel_count(x))
df_test['count_emoji'] = df_test['description'].parallel_apply(lambda x: count_emoji(x))

df_test['title_prep_l'] = df_test['title'].parallel_apply(lambda x: preprocess_text_light(x))
df_test['desc_prep_l'] = df_test['description'].parallel_apply(lambda x: preprocess_text_light(x))

CPU times: user 740 ms, sys: 7.88 s, total: 8.62 s
Wall time: 22.2 s


Let's process the text, reduce it to lemmata, and remove stop words

In [46]:
%%time 
description_words = unique_words(df[['description']])
title_words = unique_words(df[['title']])
words = list(set(description_words + title_words))

CPU times: user 1min 19s, sys: 9.08 s, total: 1min 28s
Wall time: 1min 27s


In [47]:
words[:5]

['`блудное', 'металлокопа', 'флоралшарм', 'одном', 'свитр']

In [48]:
%%time 
morph = pymorphy2.MorphAnalyzer()
norm_dict = {}

for word in words:
    if word not in norm_dict:
        norm_dict[word] = morph.parse(word)[0].normal_form

CPU times: user 2min 52s, sys: 437 ms, total: 2min 53s
Wall time: 2min 53s


In [49]:
list(norm_dict.items())[:5]

[('`блудное', '`блудный'),
 ('металлокопа', 'металлокоп'),
 ('флоралшарм', 'флоралшарм'),
 ('одном', 'один'),
 ('свитр', 'свитр')]

In [50]:
params = {
    'norm_dict': norm_dict, 
    'stop_words': stop_words}

In [51]:
%%time
df['description_prep'] = df['description'].parallel_apply(
    lambda x: preprocess_text(x, **params))
df['title_prep'] = df['title'].parallel_apply(
    lambda x: preprocess_text(x, **params))

df_test['description_prep'] = df_test['description'].parallel_apply(
    lambda x: preprocess_text(x, **params))
df_test['title_prep'] = df_test['title'].parallel_apply(
    lambda x: preprocess_text(x, **params))

CPU times: user 7.24 s, sys: 5.73 s, total: 13 s
Wall time: 4min 58s


In [52]:
X_train, X_val, y_train, y_val = train_test_split(
    df, 
    df['is_bad'], 
    test_size=0.1, 
    random_state=42,
    stratify=df['category']
    )

In [53]:
X_train['sample'] = 'train'
X_val['sample'] = 'val'
df_test['sample'] = 'test'

In [54]:
X_train.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,id,...,num_words_count,digits_count,contact_words_count,tel_count,count_emoji,title_prep_l,desc_prep_l,description_prep,title_prep,sample
336875,Капот Toyota Camry,"[арт. 43381] Капот/\n Номер производителя: 3220622610/\n/\nПодходит на Toyota Camry XV50 (Тойота, Тоета, Тойета, Камри) /\n/\n ・・・/\n/\n Артикул 43381. Пожалуйста, назовите его при звонке./\n Запчасть в наличии на собственном складе./\n/\n Дополнительную информацию по наличию и состоянию запчастей можно получить по телефону или написать сообщение в Avito, WhatsApp, Viber./\nОтправляем в регионы через Деловые линии, ПЭК, РАТЭК, Энергия, Байкал, Желдор./\n Принимаем заказы на запчасти для любых автомобилей./\n/\n Выкупаем иномарки в разбор в любом состоянии./\n Эвакуатор предоставляем./\n Принимаем авто под реализацию./\n/\n Авторазбор ""У Иваныча""/\n Крупнейший Авто-Разбор в Самарской области./\n г. Тольятти, Южное шоссе, 602Б/\n/\n График работы:/\n Пн-Сб с 9:00-69:00/\n Вс. с 9:00-67:00/\n/\nКод: 9vYsGan58z8idZ5tktE54A/\n/\n Капот.",Запчасти и аксессуары,Транспорт,15000.0,Самарская область,Тольятти,2019-07-16 14:02:15.731661,0,id_0336875,...,0,46,5,3,0,капот toyota camry,"арт. 43381 капот номер производителя: 3220622610 подходит на toyota camry xv50 (тойота, тоета, тойета, камри) ・・・ артикул 43381. пожалуйста, назовите его при звонке. запчасть в наличии на собственном складе. дополнительную информацию по наличию и состоянию запчастей можно получить по телефону или написать сообщение в avito, whatsapp, viber. отправляем в регионы через деловые линии, пэк, ратэк, энергия, байкал, желдор. принимаем заказы на запчасти для любых автомобилей. выкупаем иномарки в разбор в любом состоянии. эвакуатор предоставляем. принимаем авто под реализацию. авторазбор у иваныча крупнейший авто разбор в самарской области. г. тольятти, южное шоссе, 602б график работы: пн сб с 9:00 69:00 вс. с 9:00 67:00 код: 9vysgan58z8idz5tkte54a капот.",арт капот номер производитель подходить toyota camry xv тойота тоет тойет камри ・・・ артикул пожалуйста назвать звонок запчасть наличие собственный склад дополнительный информация наличие состояние запчасть получить телефон написать сообщение avito whatsapp viber отправлять регион деловой линия пэк ратэк энергия байкал желдора принимать заказ запчасть любой автомобиль выкупать иномарка разбор любой состояние эвакуатор предоставлять принимать авто реализация авторазбор иванович крупный авто разбор самарский область тольятти южный шоссе график работа пн сб вс код vysgan idz tkte капот,капот toyota camry,train
602099,Стартер на Ваз,Продам стартер на ваз 6110 отездел на нём пол года стартер почти новый мало б/у,Запчасти и аксессуары,Транспорт,2000.0,Калужская область,Малоярославец,2019-08-21 18:18:07.227289,0,id_0602099,...,0,4,0,0,0,стартер на ваз,продам стартер на ваз 6110 отездел на нём пол года стартер почти новый мало б у,продать стартер ваз отездеть он пол год стартер новый мало,стартер ваз,train
681935,Шкаф,"Шкаф, в хорошем состояние./\nВозможен торг./\nСвязь по номеру./\n9207473416",Мебель и интерьер,Для дома и дачи,10000.0,Белгородская область,Старый Оскол,2019-09-01 00:59:04.903949,1,id_0681935,...,0,10,0,0,0,шкаф,"шкаф, в хорошем состояние. возможен торг. связь по номеру. 9207473416",шкаф хороший состояние возможный торг связь номер,шкаф,train
92414,5-din магнитола,Продаю новый Aвтoмaгнитолa 4дин Bluetооth модуль пoзволяет coвeршать звoнки/\nне oтвлeкаясь oт вождения aвтомoбиля и прoслушивaть музыку прямо с тeлeфонa./\nFМ-тюнер RDS./\nАUХ режим./\nUSВ порт для подключения флешки./\nSD порт для подключения карты памяти./\nВыход на сабвуфер 4 ТЮЛЬПАНА./\n10wаtt на 3 колонки дают изумительное качество звучания.6 т.р,Запчасти и аксессуары,Транспорт,3000.0,Ростовская область,Ростов-на-Дону,2019-06-12 13:48:34.581693,0,id_0092414,...,0,6,0,1,0,5 din магнитола,продаю новый aвтoмaгнитолa 4дин bluetооth модуль пoзволяет coвeршать звoнки не oтвлeкаясь oт вождения aвтомoбил

In [55]:
# X_train.to_csv('{}_data/train_prep.csv'.format(FOLDER_PATH), sep='|', index=False)
# X_val.to_csv('{}_data/val_prep.csv'.format(FOLDER_PATH), sep='|', index=False)
# df_test.to_csv('{}_data/test_prep.csv'.format(FOLDER_PATH), sep='|', index=False)